In [ ]:
from selenium import webdriver
import time
import sqlite3
from bs4 import BeautifulSoup
driver = webdriver.Chrome('./chromedriver.exe')

conn = sqlite3.connect('./db.stock')
c = conn.cursor()
c.execute("CREATE TABLE IF NOT EXISTS article (id INTEGER PRIMARY KEY AUTOINCREMENT, date TEXT, time TEXT, title TEXT, content TEXT, press TEXT , stock TEXT, posi_nega TEXT)")

uri = 'https://finance.naver.com/news/market_special.nhn?&page='
# fnews = []

for page in range(5, 101):
    target = uri+str(page)
    driver.get(target)
    
    try:
        for n in range(1, 24):
#             time.sleep(2)
            driver.implicitly_wait(5)
            try: 
                driver.find_element_by_xpath('//*[@id="contentarea_left"]/div[2]/table/tbody/tr[%s]/td[1]/a' %n).click()
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                presses = soup.select('span.press > img')
                press = presses[0]['title']
                
                datas = soup.select('div.article_info > h3')
                title = datas[0].text.strip()
                
                dates = soup.select('div.article_sponsor > span')
                date = dates[0].text.strip()
                
                contents = soup.select('div#content')
                content1 = contents[0]
                for tag in content1.find_all(['li', 'h3']):
                    tag.replace_with('')
                    content = content1.text.strip()

                driver.back()
                
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                times = soup.select('table > tbody > tr:nth-child(%s) > td.wdate' %n)
                time = times[0].text.strip()[-5:]

                c.execute("INSERT INTO article( date , time , press , title, content, stock , posi_nega ) VALUES(?,?,?,?,?,?,?)",(date, time, press, title, content,'stock','posi_nega'))
                
            except:
                pass
                
    except:
        pass  

# print(fnews)
conn.commit()
c.close()

In [4]:
import pandas as pd

In [ ]:
df = pd.DataFrame(fnews, columns=['Press', 'Title', 'Content', 'Date', 'Time'])
df

In [ ]:
# df.to_csv('./saves/naver_news_data.csv', index=False, encoding='utf-8-sig')

In [5]:
conn = sqlite3.connect('./db.stock')
c = conn.cursor()
data = pd.read_sql('SELECT * FROM article', con=conn)
c.close()

data

,id,date,time,title,content,press,stock,posi_nega
0,1,2021-08-11,15:39,북한 한미연합훈련 맹비난에 경협주 하락(종합),[그래픽] 최근 남북관계 주요 일지(서울=연합뉴스) 이재윤 기자 = 김영철 노동당 ...,연합뉴스,stock,posi_nega
1,2,2021-08-11,15:36,"원티드랩, 코스닥 상장 첫날 '따상' 성공(종합)",원티드랩[프레인키웨스트 제공. 재판매 및 DB금지](서울=연합뉴스) 이미령 기자 =...,연합뉴스,stock,posi_nega
2,3,2021-08-11,14:38,가비아 COM 도메인 다년 연장 이벤트,9월 1일부터 .com 도메인 7% 인상향후 8년에 걸쳐 인상 예정[가비아 제공][...,헤럴드경제,stock,posi_nega
3,4,2021-08-11,14:37,"한솔로지스틱스, 해상운임 급등에 순익 8배…재평가 기대",한솔로지스틱스가 강세다. 2분기 실적 개선 소식이 주가에 영향을 준 것으로 보인다....,아시아경제,stock,posi_nega
4,5,2021-08-11,14:19,"엔지켐생명과학, 경구형 코로나 치료제 호주서 상용화 첫발 소식에 강세",[파이낸셜뉴스]코로나19 4차 대유행 속 게임체인저로 꼽히는 머크의 코로나19 치료...,파이낸셜뉴스,stock,posi_nega
...,...,...,...,...,...,...,...,...
75,76,2021-08-09,10:03,"코스모신소재, LG엔솔에 하이엔드 NCM 양극재 공급 추진",코스모신소재 주가가 강세다. LG에너지솔루션과 손잡고 니켈 94% 함량 NCM(니켈...,아시아경제,stock,posi_nega
76,77,2021-08-09,09:49,"카카오뱅크, MSCI 조기편입 호재…2거래일 연속 급등",카카오뱅크가 상장 이틀째에도 강세다. 모건스탠리캐피털인터내셔널(MSCI) 조기 편입...,아시아경제,stock,posi_nega
77,78,2021-08-09,09:44,"이엔코퍼레이션, 산업용 수처리 플랫폼 구축 강세",이엔코퍼레이션이 강세다. 자회사 이클린워터가 지정폐기물의 중간처분 및 폐수수탁처리 ...,아시아경제,stock,posi_nega
78,79,2021-08-09,09:41,"에코프로비엠, 주가 40만원 돌파 전망에 ‘신고가’",[이데일리 박정수 기자] 에코프로비엠(247540)이 52주 신고가를 경신했다. 증...,이데일리,stock,posi_nega
